In [3]:
# Load required modules
from spacy.matcher import Matcher
from spacy.attrs import IS_PUNCT, LOWER
import spacy
import csv
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time
import dateparser
from dateparser.search import search_dates
import requests
from dateparser import parse

nlp = spacy.load('en_core_web_lg')
matcher = Matcher(nlp.vocab)

In [2]:
def cityPattern(city):
    pattern = []
    for b in city.split():
        pattern.append({'LOWER':b})  
    return pattern

def buildPatterns(cities):
    pattern = []
    for city in cities:
        pattern.append(cityPattern(city))
    
    return list(pattern)

In [3]:
def on_match(matcher, doc, id, matches):
    return matches

def buildMatcher(patterns):
    index=0;
    for pattern in patterns:
        if(pattern):
            matcher.add(str(index),[pattern])
            index+=1
    return matcher

In [4]:
def cityMatcher(matcher, text):
    cities = []
    doc = nlp(text)
    #for token in doc:
        #print(token.text)
    matches = matcher(doc)
    for b in matches:
        match_id, start, end = b
        #print(b)
        #print (doc[start : end])
        cities.append(doc[start:end])
    return cities
    

In [5]:
stop_words = set(stopwords.words('english'))
file1 = open("placenames.txt","r")

Lines = file1.readlines()
setOfCities=set() 
count = 0
# Strips the newline character
for line in Lines:
    if(line.strip().lower() not in stop_words):
        setOfCities.add(line.strip().lower())
        #cities.append(line.strip().lower())

cities=list(setOfCities)
print(len(cities))
    

404566


In [6]:
patterns = buildPatterns(cities)
print(patterns)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
city_matcher = buildMatcher(patterns)
### Size of dictionary 
len(city_matcher)



404566

In [8]:
print(cities[13])
print(type(patterns))

thakursar
<class 'list'>


In [9]:
import datefinder

start = time.time()
matchedCities=cityMatcher(city_matcher, "Wall Crab Spider...Family Selenopidae ....Selenops or Makidopes ...Id Appreciated These spiders are very flat dorsoventrally..their running and striking speeds places them amongst the fastest in the world! Shot during the 4th SpiderIndia meet Castle Rock Karnataka Dec 19")
print(matchedCities)
end = time.time()
print(f"Runtime of the program is {end - start}")







[Castle Rock, Karnataka]
Runtime of the program is 0.6874110698699951


In [9]:
#dates extraction through spacy ner follwed by dateparser

def datesExtraction1(text):
    dates=[]
    if type(text) == str:
        doc=nlp(text)
        for ent in doc.ents:
            if(ent.label_=="DATE"):
                date=dateparser.parse(ent.text)
                if(date):
                    dateString = date.strftime("%d-%b-%Y")
                    dates.append(dateString)
    
    return dates

In [10]:
#dates extraction through only dateparser directly taling the input in the search_dates function

def datesExtraction2(text):
    dates=[]
    results=search_dates(text)
    if(results):
        for result in results:
            if(result):
                date=result[1]
                dateString=date.strftime("%d-%b-%Y")
                dates.append(dateString)
    return dates

In [4]:
def extractDates(text):
    finalDates=[]
    results=search_dates(text)
    if(results):
        for result in results:
            dateFields={}
            if(result):
                verbatimDate=result[0]
                if(parse(verbatimDate,settings={'REQUIRE_PARTS': ['day', 'month', 'year']})):
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['day', 'month', 'year']})
                    dateFields["year"]=parseDate.year
                    dateFields["month"]=parseDate.month
                    dateFields["day"]=parseDate.day
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['month', 'year']})):
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['month', 'year']})
                    dateFields["year"]=parseDate.year
                    dateFields["month"]=parseDate.month
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['day','month']})):
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['day','month']})
                    dateFields["day"]=parseDate.day
                    dateFields["month"]=parseDate.month
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['year']})):
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['year']})
                    dateFields["year"]=parseDate.year
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['month']})):
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['month']})
                    dateFields["month"]=parseDate.month
                else:
                    dateFields={}

            if(dateFields):
                finalDates.append(dateFields)                               
        return finalDates
    


    
    

In [5]:
def extract_names(namesArray):
    if(not namesArray):
        return []
        
    names=[]
    for nameObject in namesArray:
        names.append(nameObject['name'])
    return names
        

In [18]:
start = time.time()

fields=["Author","Desc","Locations","names","Dates","Permalink"]
filename = "final_data_1.csv"
faultyPermalinksFilename="Faulty_Permalinks.csv"
faultyRow=["Author","Desc","Permalink"]

with open(filename, 'a') as csvfile: 
    csvwriter = csv.writer(csvfile)         
    csvwriter.writerow(fields)
    

    


df=pd.read_csv("/home/prakhar/combine_data/spiderindia_1.csv")

for i in range(len(df)):
    try:
        text=df.loc[i,"Desc"]
        author=df.loc[i,"Author"]
        desc=df.loc[i,"Desc"]
        permalink=df.loc[i,"Permalink"]
        locations=[]
        names=[]
        dates2=[]
        if(type(desc)==str):
            locations=cityMatcher(city_matcher,desc.lower())
            #ates1=datesExtraction1(desc)
            dates2=extractDates(desc)
            query = {'text':text}
            response = requests.get('http://localhost:3006/parse',params=query)
            names=extract_names(response.json()['names'])
            #print(response.json()['names'])
        row=[author,desc,locations,names,dates2,permalink]
        with open(filename,'a') as csvfile1:
            csvwriter1=csv.writer(csvfile1)
            csvwriter1.writerow(row)
    
    except:
        print(f"value of i = {i}")
        print(f"author = {author}")
        print(f"text={text}")
        print(f"permalink = {permalink}")
        break
        
    

            
       

end = time.time()
print(f"Runtime of the program is {end - start}")


value of i = 708
author = Abhijith Apc
text=Do Jumping spider have Foresight ? Here is our latest paper on that !    Leaf suspension by a brooding female Epeus indicus (Araneae: Salticidae: Plexippina) in Karnataka Published in PECKHAMIA.  Check this… More link    https://peckhamia.com/peckhamia/PECKHAMIA_237.1.pdf  and know how a female Epeus Indicus can protect its brood .  You can also watch this youtube link for further details on same.  https://www.youtube.com/watch?v=zKWqqxYi-os ಇಂದಷ್ಟೇ ನಮ್ಮ ಒಂದು ಪ್ರಬಂಧ ಪೆಕೆಮಿಯಾ ಎಂಬ ಪತ್ರಿಕೆಯಲ್ಲಿ ಪ್ರಕಟವಾಗಿದೆ.  ವಿಷಯ :  ಜೇಡಗಳಿಗೂ ಮುಂದಾಲೋಚನೆ ಇದೆಯೇ ? ಹೌದು. ಎಪಿಯಸ್ ಎಂಬ ಜಿಗಿಯುವ ಜೇಡ , ತನ್ನ ಮರಿಗಳನ್ನು ರಕ್ಷಿಸಲು ಮಾಡಿದ ಉಪಾಯವೇ ಈ ಲೇಖನದ ತಿರುಳು.  ತನ್ನ ಮರಿಗಳಿದ್ದ ಎಲೆಯು ಹಳದಿಯಾಗಿ ಉದಿರಿ ಹೋಗುವುದು ಎಂದು ಮೊದಲೇ ಅರಿತ ಜೇಡ, ಆ ಎಲೆಯನ್ನು ಪಕ್ಕದ ಎಲೆಗೆ ಅಂಟಿಸುತ್ತದೆ. ಮುಂದೆ ಜೋರು ಗಾಳಿಗೆ ಎಲೆಯು ತೊಟ್ಟಿನಿಂದ ಕಳಚುತ್ತದೆ. ಮೊದಲೇ ಈ ಎಲೆಯನ್ನು ಭದ್ರ ಪಡಿಸಿದ್ದರಿಂದ ತನ್ನ ಸಂತತಿಗೆ ಯಾವ ಅಪಾಯವೂ  ಆಗುವುದಿಲ್ಲ.  ಅದರ ಸಂಪೂರ್ಣ ವರದಿ ಈ ಲೇಖನದಲ್ಲಿದೆ.      https://peckhamia.com/peckhamia/PECKHAMIA_237.1.pdf    ಯೂಟ್ಯೂಬಿನಲ್

In [15]:
start = time.time()

fields=["Author","Desc","Locations","names","Dates","Permalink"]
filename = "final_data_1.csv"
faultyPermalinksFilename="Faulty_Permalinks.csv"
faultyRow=["Author","Desc","Permalink"]

with open(filename, 'a') as csvfile: 
    csvwriter = csv.writer(csvfile)         
    csvwriter.writerow(fields)
    

    


df=pd.read_csv("/home/prakhar/combine_data/spiderindia_1.csv")

0
1
2
3
4
5
6
7
8
9


In [12]:
#for date extraction and conversion to a standard format
from dateparser import parse
result=parse('March', settings={'REQUIRE_PARTS': [ 'month', 'year']})
print(result.day)


AttributeError: 'NoneType' object has no attribute 'day'

In [7]:
if( parse('December 2015', settings={'STRICT_PARSING': True})==None):
    print("gotcha")

gotcha


In [6]:
#Flashtext algorithm testing

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()



stop_words = set(stopwords.words('english'))
file1 = open("extended_placenames.txt","r")

Lines = file1.readlines()
setOfCities=set() 
count = 0
# Strips the newline character
for line in Lines:
    if(line.strip().lower() not in stop_words):
        if(line.strip().lower() not in setOfCities):
            setOfCities.add(line.strip().lower())
            keyword_processor.add_keyword(line.strip().lower())
    if(line.strip().lower()=="big"):
        print(line)
            
        #cities.append(line.strip().lower())

        

cities=list(setOfCities)
print(len(cities))





BIG

1049182


In [7]:
#keywords_found = keyword_processor.extract_keywords("ID please. Nov'2021 Kohima, India.")
#print(keywords_found)



start = time.time()

fields=["Author","Desc","Locations","Scientific Names","Dates","Permalink"]
filename = "Flashtext_Locations_Dates_Names_1.csv"

faultyPermalinksFilename="Faulty_Permalinks.csv"
faultyRow=["Author","Desc","Permalink"]

with open(filename, 'a') as csvfile: 
    csvwriter = csv.writer(csvfile)         
    csvwriter.writerow(fields)
    
with open(faultyPermalinksFilename,'a') as faultyCsvFile:
    fcsvwriter=csv.writer(faultyCsvFile)
    fcsvwriter.writerow(faultyRow)

df=pd.read_csv("/home/prakhar/combine_data/spiderindia_1.csv")

for i in range(len(df)):
    try:
        text=df.loc[i,"Desc"]
        author=df.loc[i,"Author"]
        desc=df.loc[i,"Desc"]
        permalink=df.loc[i,"Permalink"]
        locations=[]
        names=[]
        dates2=[]
        if(type(desc)==str):
            locations=keyword_processor.extract_keywords(desc.lower())
            dates2=extractDates(desc)
            
            query = {'text':text}
            response = requests.get('http://localhost:3006/parse',params=query)
            if(response.json()['names']):
                names=extract_names(response.json()['names'])
            
        row=[author,desc,locations,names,dates2,permalink]
        with open(filename,'a') as csvfile1:
            csvwriter1=csv.writer(csvfile1)
            csvwriter1.writerow(row)
    
    except:
        print(f"value of i = {i}")
        print(f"author = {author}")
        print(f"text={text}")
        print(f"permalink = {permalink}")
        
        errorRow=[author,text,permalink]
        
        with open(faultyPermalinksFilename,'a') as faultyCsvFile:
            fcsvwriter=csv.writer(faultyCsvFile)
            fcsvwriter.writerow(errorRow)
        
        
        

        
end = time.time()
print(f"Runtime of the program is {end - start}")




/home/prakhar/environments/my_env/lib/python3.6/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/home/prakhar/environments/my_env/lib/python3.6/site-packages/dateparser/freshness_date_parser.py:76: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  now = self.get_local_tz().localize(now)


value of i = 708
author = Abhijith Apc
text=Do Jumping spider have Foresight ? Here is our latest paper on that !    Leaf suspension by a brooding female Epeus indicus (Araneae: Salticidae: Plexippina) in Karnataka Published in PECKHAMIA.  Check this… More link    https://peckhamia.com/peckhamia/PECKHAMIA_237.1.pdf  and know how a female Epeus Indicus can protect its brood .  You can also watch this youtube link for further details on same.  https://www.youtube.com/watch?v=zKWqqxYi-os ಇಂದಷ್ಟೇ ನಮ್ಮ ಒಂದು ಪ್ರಬಂಧ ಪೆಕೆಮಿಯಾ ಎಂಬ ಪತ್ರಿಕೆಯಲ್ಲಿ ಪ್ರಕಟವಾಗಿದೆ.  ವಿಷಯ :  ಜೇಡಗಳಿಗೂ ಮುಂದಾಲೋಚನೆ ಇದೆಯೇ ? ಹೌದು. ಎಪಿಯಸ್ ಎಂಬ ಜಿಗಿಯುವ ಜೇಡ , ತನ್ನ ಮರಿಗಳನ್ನು ರಕ್ಷಿಸಲು ಮಾಡಿದ ಉಪಾಯವೇ ಈ ಲೇಖನದ ತಿರುಳು.  ತನ್ನ ಮರಿಗಳಿದ್ದ ಎಲೆಯು ಹಳದಿಯಾಗಿ ಉದಿರಿ ಹೋಗುವುದು ಎಂದು ಮೊದಲೇ ಅರಿತ ಜೇಡ, ಆ ಎಲೆಯನ್ನು ಪಕ್ಕದ ಎಲೆಗೆ ಅಂಟಿಸುತ್ತದೆ. ಮುಂದೆ ಜೋರು ಗಾಳಿಗೆ ಎಲೆಯು ತೊಟ್ಟಿನಿಂದ ಕಳಚುತ್ತದೆ. ಮೊದಲೇ ಈ ಎಲೆಯನ್ನು ಭದ್ರ ಪಡಿಸಿದ್ದರಿಂದ ತನ್ನ ಸಂತತಿಗೆ ಯಾವ ಅಪಾಯವೂ  ಆಗುವುದಿಲ್ಲ.  ಅದರ ಸಂಪೂರ್ಣ ವರದಿ ಈ ಲೇಖನದಲ್ಲಿದೆ.      https://peckhamia.com/peckhamia/PECKHAMIA_237.1.pdf    ಯೂಟ್ಯೂಬಿನಲ್

In [28]:
if 'one' in setOfCities:
    print("present")

present
